In [1]:
%matplotlib inline

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F


import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.optim as optim
import cv2

from LFWDataset import LFWDataset
from SiameseNet import SiameseNet

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
class LFWDataset(Dataset):
    
    """Faces in the wild dataset."""

    def __init__(self, root_dir='lfw/', train=False, test=False, transform=None):
        self.train = train
        self.test = test
        self.root_dir = root_dir
        self.files = []
        self.labels = []
        self.transform = transform
        
        if (self.train and self.test) or not (self.train or self.test):
            raise ValueError('Exactly one of train and test must be set.')
        
        """
        Getting Train/Test splits
        """
        dataset = set()
        if self.train:
            filename='train.txt'
        else:
            filename='test.txt'

        with open(filename) as f:
            for line in f:
                line = line.split()
                self.files.append(line[:2])
                self.labels.append(int(line[2]))
                
        #print(self.files)
    
    def __len__(self):
        return len(self.files)
#         return sum([len(files) for r, d, files in os.walk(self.root_dir)])
#         https://stackoverflow.com/questions/16910330/return-number-of-files-in-directory-and-subdirectory

    def __getitem__(self, idx):
        im_names = self.files[idx]
        im1 = Image.open(os.path.join(self.root_dir, im_names[0]))
        im2 = Image.open(os.path.join(self.root_dir, im_names[1]))
        label = self.labels[idx]
        
        if self.transform:
            im1 = self.transform(im1)
            im2 = self.transform(im2)
            
        return im1, im2, label

In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class SiameseNet(nn.Module):

    def __init__(self):
        super(SiameseNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 5, stride=(1,1), padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d((2,2), stride=(2,2)),
            
            nn.Conv2d(64, 128, 5, stride=(1,1), padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d((2,2), stride=(2,2)),
            
            nn.Conv2d(128, 256, 3, stride=(1,1), padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d((2,2), stride=(2,2)),
            
            nn.Conv2d(256, 512, 3, stride=(1,1), padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(512 * 16 * 16, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(1024)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(2*1024, 1),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        output = self.cnn(x)
        #flatten
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output        

    def forward(self, input1, input2):
        f1 = self.forward_once(input1)
        f2 = self.forward_once(input2)
#         print(f1, f2)
        output = self.fc2(torch.cat((f1, f2), 1))
        return output


In [ ]:
%matplotlib inline
def show(img, filename=None, save=False):
    npimg = img.numpy()
    plt.axis("off")
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    plt.show()
    
    if save and filename is not None:
        plt.savefig(filename)

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

In [10]:
trainset = LFWDataset(train=True,
                      transform=transforms.Compose([transforms.Scale((128,128)),
                                                                      transforms.ToTensor()
                                                                      ]))
trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)




In [ ]:
# vis_dataloader = DataLoader(trainset,
#                         shuffle=True,
#                         num_workers=8,
#                         batch_size=8)
# dataiter = iter(vis_dataloader)

# example_batch = next(dataiter)
# concatenated = torch.cat((example_batch[0],example_batch[1]),0)

# show(torchvision.utils.make_grid(concatenated))
# print(example_batch[2].numpy())

In [2]:
net = SiameseNet().cuda()
criterion = nn.BCELoss()
learning_rate = 1e-6
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
print(net)

In [ ]:
counter = []
loss_history = [] 
iteration_number= 0
epochs = 30

In [ ]:
for epoch in range(epochs):
    for i, data in enumerate(trainloader,0):
        img0, img1 , label = data
        if i==0:
            concat = torch.cat((img0, img1), 0)
            print(concat.shape)
            grid = torchvision.utils.make_grid(concat)
            im = transforms.ToPILImage()(grid)
            plt.figure(figsize=(12,15))
            plt.imshow(im)
            plt.show()
#             im.save("output.png")
        img0, img1 , label = Variable(img0).cuda(), Variable(img1).cuda() , Variable(label).cuda()
        output = net(img0,img1)
        label = label.unsqueeze(1).float()
        if i==0:
            print(output, label)
        
        optimizer.zero_grad()
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss.data[0]))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss.data[0])

In [ ]:
show_plot(counter,loss_history)

In [ ]:
torch.save(net.state_dict(), "model")

In [5]:
testset = LFWDataset(test=True,
                     transform=transforms.Compose([transforms.Scale((128, 128)),
                                                                      transforms.ToTensor()
                                                                      ]))
testloader = DataLoader(testset, batch_size=8, shuffle=True, num_workers=2)

In [6]:
net.load_state_dict(torch.load('p1ai'))

In [11]:
right=wrong=0.

for i, data in enumerate(testloader,0):
    img0, img1, label = data    
    img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()
    
    output = net(img0,img1)
    for x,y in zip(output, label):
#         print(x.data[0], y.data[0])
        if (x.data[0]<=0.5 and y.data[0]==0) or (x.data[0]>0.5 and y.data[0]==1):
            right+=1
        else:
            wrong+=1
            

In [12]:
#     print(right, wrong)
testacc = right/(right+wrong)
print("Accuracy on test set: {:.2f}".format(testacc))

Accuracy on test set: 0.55


In [13]:
right=wrong=0.

for i, data in enumerate(trainloader,0):
    img0, img1, label = data    
    img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()
    
    output = net(img0,img1)
    for x,y in zip(output, label):
#         print(x.data[0], y.data[0])
        if (x.data[0]<=0.5 and y.data[0]==0) or (x.data[0]>0.5 and y.data[0]==1):
            right+=1
        else:
            wrong+=1
            

In [14]:
#     print(right, wrong)
trainacc = right/(right+wrong)
print("Accuracy on train set: {:.2f}".format(trainacc))

Accuracy on train set: 0.99


In [ ]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [ ]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.autograd.Variable(torch.randn(3), requires_grad=True)
target = torch.autograd.Variable(torch.FloatTensor(3).random_(2))
print(input, target)
output = loss(m(input), target)
output.backward()